In [1]:
# import relevant modules/packages
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pgmpy.estimators import K2Score, BicScore#, BDeuScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, BayesianEstimator

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

from pgmpy.estimators import BdeuScore, K2Score, BicScore
#from pgmpy.models import BayesianModel

from pgmpy.estimators import HillClimbSearch

from pgmpy.estimators import BayesianEstimator

from pgmpy.inference import VariableElimination

## Input Data as a df

In [16]:
Data=pd.read_csv("DataInputFinal.csv")

## Discretize data to create binned columns, and create new df with just bin columns

In [17]:
import warnings
warnings.filterwarnings("ignore")

Data['Asthma_binned'] = pd.cut(Data['Asthma Hospitalizations (Adults) by NTA'], bins=3, labels=False)
Data['FamiliesIncome_binned'] = pd.cut(Data['Families with income < $10,000'], bins=3, labels=False)
Data['11th_binned'] = pd.cut(Data['Number of persons 25+ yrs in 11th grade'], bins=3, labels=False)
Data['ChildrenPoverty'] = pd.cut(Data['Children under 5 years old in Poverty'], bins=3, labels=False)
Data['PercentBlack_binned'] = pd.cut(Data['Percent Black Alone'], bins=3, labels=False)
Data['10th_binned'] = pd.cut(Data['Number of persons 25+ yrs in 10th grade'], bins=3, labels=False)
Data['Poverty_binned'] = pd.cut(Data['Poverty'], bins=3, labels=False)
Data['Families125K_binned'] = pd.cut(Data['Families with income between $125,000-$149,999'], bins=3, labels=False)
Data['Unemployment_binned'] = pd.cut(Data['Unemployment'], bins=3, labels=False)
Data['Families150K_binned'] = pd.cut(Data['Families with income between $150,000-$199,999'], bins=3, labels=False)
Data['PercentAsian_binned'] = pd.cut(Data['Percent Asian Alone'], bins=3, labels=False)



Data_cutfeatures = Data.iloc[:,64:]


Data_cutfeatures.head()

,Asthma_binned,FamiliesIncome_binned,11th_binned,ChildrenPoverty,PercentBlack_binned,10th_binned,Poverty_binned,Families125K_binned,Unemployment_binned,Families150K_binned,PercentAsian_binned
0,0,0,0,0,1,0,0,1,1,1,0
1,0,0,0,0,0,0,0,2,0,2,0
2,0,0,0,0,0,0,0,2,0,2,0
3,0,0,0,0,0,0,0,1,0,1,0
4,0,0,0,0,0,0,0,1,0,2,1


In [18]:
# train test split
train,test=train_test_split(Data_cutfeatures,random_state=9,test_size=0.3)

## Perform Hillclimb and create 'best model', with resulting edges

In [12]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = BayesianModel(hc.estimate().edges())
print(best_model.edges())

[('11th_binned', 'Poverty_binned'), ('11th_binned', '10th_binned'), ('Asthma_binned', 'Unemployment_binned'), ('Unemployment_binned', 'PercentBlack_binned'), ('ChildrenPoverty', 'Families150K_binned'), ('Families150K_binned', 'Families125K_binned'), ('FamiliesIncome_binned', 'Asthma_binned'), ('Poverty_binned', 'FamiliesIncome_binned'), ('Poverty_binned', 'ChildrenPoverty')]


In [13]:
from pgmpy.estimators import BayesianEstimator
# your code here
# best_model = BayesianModel([('11th_binned', 'Asthma_binned'), ('10th_binned', '11th_binned'), ('Asthma_binned', 'PercentBlack_binned'),
#                             ('Asthma_binned', 'Unemployment_binned'), ('Asthma_binned', 'ChildrenPoverty'),
#                             ('ChildrenPoverty', 'Poverty_binned'),
#                             ('ChildrenPoverty', 'Families150K_binned'),
#                             ('Families150K_binned', 'Families125K_binned'),
#                             ('Poverty_binned', 'FamiliesIncome_binned')])
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print('CPD of {variable}:'.format(variable=cpd.variable))
    print(cpd)
print best_model.local_independencies(best_model.nodes())

CPD of PercentBlack_binned:
+------------------------+------------------------+------------------------+------------------------+
| Unemployment_binned    | Unemployment_binned(0) | Unemployment_binned(1) | Unemployment_binned(2) |
+------------------------+------------------------+------------------------+------------------------+
| PercentBlack_binned(0) | 0.9102564102564102     | 0.4782608695652174     | 0.6                    |
+------------------------+------------------------+------------------------+------------------------+
| PercentBlack_binned(1) | 0.0641025641025641     | 0.2826086956521739     | 0.1                    |
+------------------------+------------------------+------------------------+------------------------+
| PercentBlack_binned(2) | 0.02564102564102564    | 0.2391304347826087     | 0.3                    |
+------------------------+------------------------+------------------------+------------------------+
CPD of 11th_binned:
+----------------+----------+
| 11

## Perform prediction onto test data

In [15]:
from collections import defaultdict

def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

train_x = train[['FamiliesIncome_binned','11th_binned','ChildrenPoverty', 'PercentBlack_binned', '10th_binned',
                'Poverty_binned', 'Families125K_binned', 'Unemployment_binned','Families150K_binned']]
# #predicted_train = best_model.predict(train_x) should work, but is buggy in the current version
# predicted_train = temp_predict(best_model, train_x)
# print "In sample:",(train.loc[:,'A']==predicted_train['A']).mean()

test_x = test[['FamiliesIncome_binned','11th_binned','ChildrenPoverty', 'PercentBlack_binned', '10th_binned',
                'Poverty_binned', 'Families125K_binned', 'Unemployment_binned','Families150K_binned']]
#predicted_test = best_model.predict(test_x) #should work, but is buggy in the current version
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(test.loc[:,'Asthma_binned']==predicted_test['Asthma_binned']).mean()

Out of sample: 0.8703703703703703
